# Display friendly type names
Using a homemade library (https://github.com/pthom/type_name). 

In [1]:
#include "typ.cpp"
#include "wip_lambda.cpp"

In [2]:
std::cout << Typename_DefaultConstructibleType<std::string>();

Full : const std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >
Clean: const std::string


In [3]:
std::cout << Typename_DefaultConstructibleType<std::vector<int>>();

Full : const std::vector<int, std::allocator<int> >
Clean: const std::vector<int>


In [4]:
{
auto f = [](int & a, int & b)  { return std::pair<int, double>(a + b, cos(a + b)); };
std::cout << type_lambda(f) << "\n";
}

lambda: (int&, int&) -> std::pair<int, double> 


In [5]:
{
auto f = [](int & a, int & b)  { return std::pair<int, double>(a + b, cos(a + b)); };
std::cout << type_lambda(f) << "\n";
}

lambda: (int&, int&) -> std::pair<int, double> 


In [6]:
#pragma cling add_include_path("../../external/FunctionalPlus/include")
#include <fplus/fplus.hpp>

In [7]:
{
    auto my_square = [](int a) { return a * a;};
    auto my_double = [](int a) { return a * 2; };
    auto ff = fplus::fwd::compose(my_square, my_double);
    auto as_mem_fn = std::mem_fn( & decltype(ff)::operator()<int> );
    std::cout << var_type_name_full(as_mem_fn) << '\n';
    
    std::cout << '\n';
    std::cout << type_lambda_polymorphic<decltype(ff), int>(ff);
}

std::_Mem_fn<int (fplus::fwd::internal::compose_helper<__cling_Un1Qu36(void*)::$_0, __cling_Un1Qu36(void*)::$_1>::*)(int&&) const>

lambda: (int&&) -> int 

In [8]:
int my_function(int a, double b)
{
    return a + static_cast<int>(b);
}

In [9]:
auto my_lambda = [](int a, double b)
{
    return a + static_cast<int>(b);
};

In [1]:
#include "wip_lambda.cpp"
test_lambdas()

lambda: (int&, int&) -> std::pair<int, double> 
std::_Mem_fn<int (fplus::fwd::internal::compose_helper<test_lambdas()::$_1, test_lambdas()::$_2>::*)(int&&) const>

lambda: (int&&) -> int 

In [2]:
{
    std::cout << type_lambda_polymorphic<decltype(fplus::split), std::string>(ff);
}

input_line_9:3:51: error: reference to overloaded function could not be resolved; did you mean to call it?
    std::cout << type_lambda_polymorphic<decltype(fplus::split), std::string>(ff);
                                                  ^~~~~~~~~~~~
../../external/FunctionalPlus/include/fplus/split.hpp:391:6: note: possible target for call
auto split(const T& x, bool allow_empty, const ContainerIn& xs)
     ^


Interpreter Error: 